#### Ejercicio 1: "Re-implementando" (opcional):

Copiar esta notebook y borrar las celdas donde creamos la red neuronal (desde la sección "Creando nuestra red neuronal" en adelante). De modo que sólo quede la parte donde importamos el dataset. Y re-escribir:
- La creación de los pesos y la función que dado las features produce la predicción.
- La función de costo (negative log likehood).
- La métrica: accuracy.
- El loop de entrenamiento, incluyendo código para evaluar en cada época y mostrar cómo mejora la métrica.
- Un ejemplo de cómo se usa el modelo para inferencia (sobre un sólo caso, para calcular la predicción).

In [2]:
import torch
import numpy as np
from matplotlib import pyplot
import math

from pathlib import Path
import requests

import pickle
import gzip

In [3]:

DATA_PATH = Path("../data")
PATH = DATA_PATH / "mnist"

# Creamos el directorio si no existe
PATH.mkdir(parents=True, exist_ok=True)

URL = "https://github.com/pytorch/tutorials/raw/main/_static/"
FILENAME = "mnist.pkl.gz"

# Descargamos mnist.pkl.gz utilizando un HTTP GET request
if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)

In [4]:
# Los arrays de las imagenes fueron guardados en un archivo formato pickle, que se utiliza para persistir variable en Python
with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

((50000, 784), (50000,), (10000, 784), (10000,))

In [5]:
# Funcion para obtener un batch de imagenes
def get_batch(a, i):
    return torch.tensor(a[batch_size*i:batch_size*(i+1),...])


In [7]:
# Funcion de prediccion de la red
def calculate_predictions(t):
    linear_combination_1 = t @ W_1 + b_1
    hidden_layer_1 = torch.max(torch.tensor(0), linear_combination_1)
    
    linear_combination_2 = hidden_layer_1 @ W_2 + b_2
    hidden_layer_2 = torch.max(torch.tensor(0), linear_combination_2)

    linear_combination_3 = hidden_layer_2 @ W_3 + b_3

    return softmax(linear_combination_3)

In [8]:
# Funcion para inicializar los pesos de la red
def w_rand(n_in, n_out):
    return torch.normal(0, math.sqrt(6) / math.sqrt(n_in + n_out), size=(n_in,n_out), requires_grad=True)


In [9]:
# Funcion softmax
def softmax(x):
    return x.exp() / x.exp().sum(-1).unsqueeze(-1)

In [10]:
# Funcion de perdida logaritmica
def nnl(pred_batch, target_batch):
    bz = pred_batch.shape[0]
    return -torch.log(pred_batch[range(bz), target_batch]).mean()

In [11]:
# funcion que calcula el accuracy de un batch de imagenes
def accuracy(probs, target):
    class_predictions = torch.argmax(probs, dim=1)
    return (class_predictions == target).float().mean()

In [12]:
# Inicializamos los pesos de la red
W_1 = w_rand(28*28, 512)
b_1 = torch.zeros(512, requires_grad=True)

W_2 = w_rand(512, 512)
b_2 = torch.zeros(512, requires_grad=True)

W_3 = w_rand(512,10)
b_3 = torch.zeros(10, requires_grad=True)

weights = [W_1, b_1, W_2, b_2, W_3, b_3]

In [13]:
batch_size = 32
lr = 0.01
epochs = 10

#calculo la cantidad de batches
n_batches_train = x_train.shape[0] // batch_size
n_batches_valid = (x_valid.shape[0] + batch_size - 1) // batch_size


In [15]:

for epoch in range(epochs):
    lost_train_sum = 0

    # Entrenamiento
    for i in range(n_batches_train):
        x_train_batch = get_batch(x_train, i)
        y_train_batch = get_batch(y_train, i)
    
        # Calcular la prediccion del minibatch
        preds = calculate_predictions(x_train_batch)

        # Calcular la loss
        loss = nnl(preds, y_train_batch)

        # Actualizar los pesos
        for w in weights:
            if w.grad is not None:
                w.grad.zero_()
        
        # Calcular el gradiente
        loss.backward()

        lost_train_sum += loss.item()

        # actualizar el peso
        with torch.no_grad():
            for w in weights:
                w -= w.grad * lr

    # Volver a obtener un rango de batches random
    permutation = torch.randperm(x_train.shape[0])

    x_train = x_train[permutation,...]
    y_train = y_train[permutation,...]

    # Evaluamos los datos en validación
    loss_validation_sum = 0
    accuracy_sum = 0

    for i in range(n_batches_valid):
        x_batch = get_batch(x_valid, i)
        y_batch = get_batch(y_valid, i)

        # Calcular la prediccion del minibatch
        preds = calculate_predictions(x_batch)

        # Calcular la loss
        loss = nnl(preds, y_batch)

        loss_validation_sum += loss.item()

        # Calcular el accuracy
        accuracy_sum += accuracy(preds, y_batch)

    print(f"Epoch {epoch} - Train loss: {lost_train_sum/n_batches_train} - Validation loss: {loss_validation_sum/n_batches_valid} - Validation accuracy: {accuracy_sum/n_batches_valid}")


Epoch 0 - Train loss: 0.06399812686904578 - Validation loss: 0.09806575496181155 - Validation accuracy: 0.9725439548492432
Epoch 1 - Train loss: 0.058486916950162116 - Validation loss: 0.10016746765274864 - Validation accuracy: 0.972244381904602
Epoch 2 - Train loss: 0.05349240983343265 - Validation loss: 0.09314777689800707 - Validation accuracy: 0.9738418459892273
Epoch 3 - Train loss: 0.04941959250633391 - Validation loss: 0.0931308099835629 - Validation accuracy: 0.9730431437492371
Epoch 4 - Train loss: 0.045496526886303595 - Validation loss: 0.08948774709099958 - Validation accuracy: 0.9739416837692261
Epoch 5 - Train loss: 0.04211268131218722 - Validation loss: 0.0871676202356137 - Validation accuracy: 0.9750399589538574
Epoch 6 - Train loss: 0.03856687843252543 - Validation loss: 0.08874176073850641 - Validation accuracy: 0.9739416837692261
Epoch 7 - Train loss: 0.03580513498520779 - Validation loss: 0.08789845608207716 - Validation accuracy: 0.9746405482292175
Epoch 8 - Train l

#### Ejercicio 2: "Analizando nuestro modelo" (obligatorio)

Usando como base alguno de los dos ejemplos realizar código que:
- Calcule la matriz de confusión para los dígitos. La matriz de confusión es una métrica, por lo que debe ser calculada sólo en validación.
- Utilizando la matriz de confusión elegir el par de dígitos dónde el modelo se confunde más y mostrar 20 ejemplos mal clasificados.
- Una función que muestre los 20 ejemplos de validación con mayor costo. Intuitivamente estos son los ejemplos donde el modelo está más errado: arroja probabilidades altas para clases que no son la correcta.
- Una función que muestre los 20 ejemplos de validación donde el modelo arroja probabilidades más bajas. Esto se puede interpretar como que el modelo está "poco seguro" para estos casos.

Todo este trabajo de visualización suele ser comun realizarlo cuando se trabaja con datasets reales para limpiar el dataset de ejemplos malformados, maletiquetados, etc.
En caso de utilizarlo para limpiar el dataset hay que realizarlo sobre todo el conjunto de datos (no sólo validación). Y es importante que el modelo no llegue a sobre-ajustar ni un poco (ya que usaremos por ejemplo el costo en training para limpiar el dataset).

In [25]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Creo un array para guardar los resultados
results = []

# Crear la matriz de confusión
confusion_matrix = torch.zeros((10, 10))

# recorrer los batches de validación
for i in range(n_batches_valid):
    x_batch = get_batch(x_valid, i)
    y_batch = get_batch(y_valid, i)

    # Calcular las predicciones de los datos de validación
    preds = calculate_predictions(x_batch)

    # obtener la clase predicha
    predicted_labels = torch.argmax(preds, dim=1)

    #guardar los preds, los valores reales y las probabilidades
    results.append((y_batch.numpy().tolist()
                    , predicted_labels.numpy().tolist()
                    , preds.detach().numpy().max(axis=1)
    ))
  
    # Actualizar la matriz de confusión
    for j in range(len(y_batch)):
        confusion_matrix[y_batch[j], predicted_labels[j]] += 1
    

Matriz de confusión

In [26]:
# Convertir a un pandas DataFrame
confusion_df = pd.DataFrame(confusion_matrix.numpy(), columns=range(10), index=range(10))
print(confusion_df)

       0       1      2      3      4      5      6       7      8      9
0  974.0     0.0    8.0    0.0    0.0    1.0    3.0     0.0    2.0    3.0
1    0.0  1056.0    2.0    2.0    0.0    0.0    1.0     1.0    2.0    0.0
2    1.0     2.0  966.0    3.0    1.0    1.0    3.0     7.0    6.0    0.0
3    0.0     0.0    6.0  999.0    0.0   13.0    0.0     2.0    6.0    4.0
4    0.0     5.0    0.0    0.0  963.0    0.0    2.0     2.0    1.0   10.0
5    4.0     1.0    2.0   17.0    3.0  864.0   12.0     1.0    7.0    4.0
6    4.0     1.0    0.0    0.0    1.0    2.0  957.0     0.0    2.0    0.0
7    1.0     2.0    4.0    2.0    2.0    0.0    0.0  1071.0    1.0    7.0
8    0.0     4.0    4.0    8.0    0.0    4.0    1.0     1.0  981.0    6.0
9    3.0     3.0    0.0    6.0    8.0    2.0    0.0    11.0    4.0  924.0


Los 2 digitos que más se confunden

In [21]:
# Utilizando la matriz de confusión obtenemos los valores que más se confunden
max_values = confusion_df.where(~np.eye(confusion_df.shape[0], dtype=bool)).max().max()

# Obtengo los indices de los valores máximos
indices = np.where(confusion_df == max_values)

# Obtengo los valores de los indices
digit1, digit2 = indices[0][0], indices[1][0]

digit1, digit2

(5, 3)

Los 20 primeros valores que son mal predichos

In [23]:
# Convertor en un DataFrame
df_results = pd.DataFrame(data=[(y, yhat, prob) for r in results for y, yhat, prob in zip(r[0], r[1], r[2])], columns=["y", "yhat", "prob"])

# Agregar columna de aciertos
df_results["correct"] = df_results["y"] == df_results["yhat"]

# Filtar los 20 priemros no aciertos
df_results[~df_results["correct"]].head(20)

,y,yhat,prob,correct
91,5,4,0.547112,False
192,0,6,0.634591,False
212,3,9,0.511139,False
239,9,0,0.870687,False
246,8,7,0.547771,False
317,3,8,0.611102,False
318,9,7,0.532514,False
320,5,7,0.526388,False
322,5,3,0.577177,False
329,9,7,0.994454,False


Función que muestra los 20 ejemplos de validación con mayor costo

In [29]:
def top_loss(df, n=20):
    return df.sort_values("prob", ascending=False).head(n)

top_loss(df_results[~df_results["correct"]], 20)

,y,yhat,prob,correct
3507,5,8,0.999367,False
9915,4,7,0.999174,False
7662,5,6,0.999036,False
1944,4,9,0.998715,False
9731,5,6,0.996954,False
9719,9,5,0.995817,False
3216,9,0,0.995762,False
1248,9,4,0.994682,False
329,9,7,0.994454,False
2241,8,9,0.991577,False


Casos donde el modelo arroja las probabilidades mas bajas sobre los aciertos

In [34]:
def menos_seguros(df, n=20):
    return df.sort_values("prob", ascending=True).head(n)

menos_seguros(df_results[df_results["correct"]], 20)

,y,yhat,prob,correct
120,1,1,0.297857,True
2886,7,7,0.341790,True
6081,1,1,0.345887,True
2895,5,5,0.375313,True
3978,5,5,0.387597,True
6586,5,5,0.387770,True
5294,4,4,0.409243,True
6939,5,5,0.419619,True
7718,7,7,0.430644,True
6838,5,5,0.431342,True


#### Ejercicio 3: "Fine-tunning" (opcional)

Re-escribir el código de la unidad anterior utilizando el modelo resnet34 con pesos entrenados para ImageNet.
**Si deciden hacer este ejercicios las siguientes secciones les serán útiles**

ResNet es una familia de arquitecturas de redes convolucionales.

In [8]:
import torchvision, torch
from torchvision import datasets, transforms, models
from torch import nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet_model = torchvision.models.resnet34(pretrained=True)
resnet_model.fc = nn.Linear(512, 10)
resnet_model.to(device)
resnet_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [85]:
from torch.utils.data import TensorDataset, DataLoader

# Convierto a tensores de 28x28
train_new = torch.tensor(x_train.reshape(-1,28,28))
valid_new = torch.tensor(x_valid.reshape(-1,28,28))

# Apilo 3 veces para tener 3 canales
train_new = torch.stack([train_new,train_new,train_new]).permute(1,0,2,3)
valid_new = torch.stack([valid_new,valid_new,valid_new]).permute(1,0,2,3)

# Verifico las dimensiones
print(train_new.shape, valid_new.shape)

 # Armo los dataset con las imagenes y las etiquetas
train_dataset_new  = TensorDataset(train_new, torch.tensor(y_train))
valid_dataset_new  = TensorDataset((valid_new), torch.tensor(y_valid))

# Creo los dataloaders
train_dataloader_new = DataLoader(train_dataset_new, batch_size=64, shuffle=True)
valid_dataloader_new = DataLoader(valid_dataset_new, batch_size=64, shuffle=True)

# Verifico la cantidad de batches
print(len(train_dataloader_new), len(valid_dataloader_new))

torch.Size([50000, 3, 28, 28]) torch.Size([10000, 3, 28, 28])
782 157


Entrenar el modelo solo en la ultima capa lineal que fue modificada

In [86]:
# Establecer el atributo requires_grad de todos los parámetros en el modelo en False.
for param in resnet_model.parameters():
    param.requires_grad = False

# Establecer el atributo requires_grad de los parámetros de la ultima capa en el modelo en True.
for param in resnet_model.fc.parameters():
    param.requires_grad = True

# Definir funciones de perdida y optimizador
loss_func_fc = torch.nn.CrossEntropyLoss()

# Solo le paso los parametros de la ultima capa al optimizador
optimizer = torch.optim.Adam(resnet_model.fc.parameters(), lr=0.01)

# Entrenar el modelo en pocas epocas (3)
for epoch in range(3):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader_new, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = resnet_model(inputs)
        loss = loss_func_fc(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

    # Guardar el modelo entrenado
torch.save(resnet_model.state_dict(), "resnet_model_new.pth")

[1,   200] loss: 1.754
[1,   400] loss: 1.592
[1,   600] loss: 1.677
[2,   200] loss: 1.689
[2,   400] loss: 1.705
[2,   600] loss: 1.616
[3,   200] loss: 1.690
[3,   400] loss: 1.745
[3,   600] loss: 1.723
[4,   200] loss: 1.723
[4,   400] loss: 1.700
[4,   600] loss: 1.718
[5,   200] loss: 1.740
[5,   400] loss: 1.732
[5,   600] loss: 1.841


In [70]:
# Cargar el modelo guardado
resnet_model_fc_modified.load_state_dict(torch.load("resnet_model_fc_modified.pth"))

NameError: name 'resnet_model_fc_modified' is not defined

In [88]:
# Volver a establecer el atributo requires_grad de todos los parámetros en el modelo en True para entrenar el modelo completo.
for param in resnet_model.parameters():
    param.requires_grad = True

n_epochs = 10
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet_model_fc_modified.parameters(), lr=0.01)

for idx_epoch in range(n_epochs):
    # Loop de entrenamiento
    loss_train_sum = 0
    n_batches_train = 0

    for x_train_batch, y_train_batch in train_dataloader_new:
        predictions = resnet_model(x_train_batch)
        loss = loss_func(predictions, y_train_batch)
        loss_train_sum += loss.item()
        n_batches_train += 1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Evaluamos los datos en validación
    loss_validation_sum = 0
    accuracy_sum = 0
    for x_valid_batch, y_valid_batch in valid_dataloader_new:
        predictions = resnet_model(x_valid_batch)
        loss = loss_func(predictions, y_valid_batch)
        loss_validation_sum += loss.item()
        accuracy_sum += accuracy(predictions, y_valid_batch).item()
    
    # Imprimimos el loss en train y validación y la métrica (siempre en validación)
    accuracy_validation = accuracy_sum / n_batches_valid
    loss_validation = loss_validation_sum / n_batches_valid
    train_validation = loss_train_sum / n_batches_train
    print(f'epoch {idx_epoch} | train loss {loss_validation} | validation loss {train_validation} | accuracy {accuracy_validation}')

    # Guardar el modelo entrenado
torch.save(resnet_model.state_dict(), "resnet_model_new.pth")

epoch 0 | train loss 0.6412478035059981 | validation loss 0.6130310084630289 | accuracy 0.3863817891373802
epoch 1 | train loss 0.0475972165270283 | validation loss 0.225638683009988 | accuracy 0.4884185303514377
epoch 2 | train loss 0.03259653347777203 | validation loss 0.08496644936920003 | accuracy 0.4917132587859425
epoch 3 | train loss 0.030387666148762888 | validation loss 0.0741072650503098 | accuracy 0.4932108626198083
epoch 4 | train loss 0.4455229650480679 | validation loss 0.05770228753882267 | accuracy 0.4397464057507987
epoch 5 | train loss 0.028562948756372206 | validation loss 0.1412349658856845 | accuracy 0.4926617412140575
epoch 6 | train loss 0.028841269691623508 | validation loss 0.055586384613658814 | accuracy 0.49291134185303515
epoch 7 | train loss 0.028345280656436333 | validation loss 0.049346158851731256 | accuracy 0.49346046325878595
epoch 8 | train loss 0.023960166927665092 | validation loss 0.0419473947392648 | accuracy 0.49460862619808305
epoch 9 | train lo

Finalmente podemos calcular la predicción del modelo:

In [99]:
logprobs = resnet_model(x_valid_batch)
logprobs.shape, logprobs[0].sum()

(torch.Size([16, 10]), -16.87685775756836)

Esta última capa es completamente nueva. Los pesos son completamente aleatorios. Cuándo esto sucede lo recomendable es entrenar algunas épocas esta capa solamente y luego entrenar toda la red. Esto se puede lograr pasando al optimizer sólo los parámetros de ésta última capa. Y luego crear otro optimizer y pasarle todos los parámetros del modelo.

#### Ejercicio 4: "Dataset propio" (opcional)

Crear un dataset de imágenes de tu interés y entrenar un clasificar ResNet (ver ejercicio anterior) en dicho dataset.

Para crear un dataset se puede utilizar, por ejemplo la API de Bing para descargar automáticamente imágenes del resultado de una búsqueda: https://pub.aimind.so/build-your-dataset-with-bing-search-api-1adf6b550a3c

#### Ejercicio 5: "Regresión sobre imágenes" (opcional)

Hacer un modelo basado en ResNet que prediga la posición de la cabeza dada una imágen.
**Este es, probablemente, el ejercicio más difícil del TP.**
Si alguien está dispuesto y comienza a hacerlo, le sugiero ir poniéndo sus avances en el foro para que lo pueda ir guiando.
A grandes rasgos los pasos debieran ser:
1. Implementar un dataset particular. E
2. Las imágenes no tienen el mismo size. Para poder construir los batches, necesitamos que lo sean (todas las dimensiones deben ser iguales). Va a ser necesario aplicar recorte (cropp) y reescalamiento (resize). Estas transformaciones se suelen aplicar como transformaciones que aplica la clase Dataset implementada. Notar que los target (las coordenadas de la cabeza) cambian cuando se recorta una imágen. O sea que también hay que aplicar transformaciones sobre los target.
3. El resto es similar al ejercicio anterior: hay que tomar un modelo ResNet ya entrenado (puede ser resnet34) y cambiarle la última capa para que calcule dos números: las dos coordenadas. Como ahora la salida no es una distribución de probabilidad no debemos aplicar softmax sobre la salida. Algo que podemos hacer es aplicar la función sigmoide a cada una de las salidas y escalarla, es una práctica habitual para problemas de regresión en un rango acotado.
4. Como función de costo ahora tendremos MSE (que aplicaremos sobre cada coordenada).